In [2]:
#webcam 작동 여부 read 
cam = cv2.VideoCapture(0) # video capture
while True:
    _, frame = cam.read()
    cv2.imshow('Camera feed', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): #q누르면 꺼짐
        break
cam.release() #카메라 설정권한 on해주셈 

In [1]:
from imutils.video import VideoStream
import cv2
import time
import f_detector #만든 detector 라이브러리 
import imutils 
import numpy as np
import os

In [2]:
detector = f_detector.eye_blink_detector() 
# initial 변수 
COUNTER = 0 
TOTAL = 0 
con = False

if con == True:
    t_data = './data/test1.mp4'
    vs = VideoStream(src = t_data).start() 
else:
    t_data = 'Real Time'
    vs = VideoStream(src = 0).start() 
    

print(f'data con : {t_data}')
# video 측정 
while True:
    start_time = time.time()
    im = vs.read() 
    im = cv2.flip(im, 1)
    im = imutils.resize(im, width = 700)
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY) 

    #detector ratio 측정
    rectangles = detector.detector_faces(gray, 0)
    boxes_face = f_detector.convert_rectangles2array(rectangles, im)
    if len(boxes_face) != 0: # 사람 감지시
        #eye select with mask 
        areas = f_detector.get_areas(boxes_face)
        index = np.argmax(areas)
        rectangles = rectangles[index]
        boxes_face = np.expand_dims(boxes_face[index], axis = 0)
        # 깜빡임 detector
        COUNTER, TOTAL = detector.eye_blink(gray, rectangles, COUNTER, TOTAL)
        # indicate bounding box
        img_post = f_detector.bounding_box(im, boxes_face, ['blinks: {}'.format(TOTAL)])
    else:
        img_post = im 
    
    # visualization
    end_time = time.time() - start_time 
    FPS = 1/end_time 
    cv2.putText(img_post,f"FPS: {round(FPS,3)}",(10,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
    cv2.imshow('blink_detection',img_post)
    if cv2.waitKey(1) &0xFF == ord('q'):
        break

data con : Real Time
